#### IMPORTO LIBRERIAS ####

In [92]:
import pandas as pd
import numpy as np
from sklearn import linear_model

#configuro notebook para mostrar varios salidas en la misma celda
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#amplio el output de columnas a 500
pd.set_option('display.max_columns', 500)

!pwd
!ls -l

/Users/mac/Documents/MASTER_DATA_SCIENCE/tfm
total 345384
-rw-r--r--   1 mac  staff     75498 18 jun 01:42 ANALISIS_DGT.ipynb
-rw-r--r--   1 mac  staff  41178839 17 jun 00:18 Data_cleaned.zip
-rw-r--r--   1 mac  staff    104803 15 abr 11:08 TFM_DGT-pruebas.ipynb
-rw-r--r--   1 mac  staff    160556 18 jun 01:44 TFM_DGT.ipynb
-rw-r--r--   1 mac  staff  33726464 31 may 22:31 aa.db
drwxr-xr-x  22 mac  staff       748 17 jun 22:02 cleaned
-rw-r--r--   1 mac  staff  33726464 31 may 14:35 data.db
-rw-r--r--   1 mac  staff      8192 31 may 13:43 datos_dgt.db
-rw-r--r--   1 mac  staff         0 23 may 22:42 gg
-rw-r--r--   1 mac  staff  67846144 31 may 22:27 jj.db
drwxr-xr-x   5 mac  staff       170 30 mar 23:28 metadata
drwxr-xr-x  11 mac  staff       374 30 mar 23:28 raw
drwxr-xr-x  23 mac  staff       782 16 jun 23:17 raw_zip


### CONSERVO LOS DATOS ORIGINALES EN LA CARPETA RAW. LOS FICHEROS DE TRABAJO ESTAN EN LA CARPETA CLEANED

In [93]:
!ls -l
!cp -r raw/* cleaned/

total 345384
-rw-r--r--   1 mac  staff     75498 18 jun 01:42 ANALISIS_DGT.ipynb
-rw-r--r--   1 mac  staff  41178839 17 jun 00:18 Data_cleaned.zip
-rw-r--r--   1 mac  staff    104803 15 abr 11:08 TFM_DGT-pruebas.ipynb
-rw-r--r--   1 mac  staff    160556 18 jun 01:44 TFM_DGT.ipynb
-rw-r--r--   1 mac  staff  33726464 31 may 22:31 aa.db
drwxr-xr-x  22 mac  staff       748 17 jun 22:02 cleaned
-rw-r--r--   1 mac  staff  33726464 31 may 14:35 data.db
-rw-r--r--   1 mac  staff      8192 31 may 13:43 datos_dgt.db
-rw-r--r--   1 mac  staff         0 23 may 22:42 gg
-rw-r--r--   1 mac  staff  67846144 31 may 22:27 jj.db
drwxr-xr-x   5 mac  staff       170 30 mar 23:28 metadata
drwxr-xr-x  11 mac  staff       374 30 mar 23:28 raw
drwxr-xr-x  23 mac  staff       782 16 jun 23:17 raw_zip


### GUARDO UN LISTADO POR TIPOLOGIA DE FICHERO, PARA POSTERIORMENTE ALMACENARLO EN UN DICCIONARIO ###

In [94]:
accvict=!ls ./cleaned/MICRODATOS_ACC_VICT_20*/TABLA_ACCVICT_*.csv
pers=!ls ./cleaned/MICRODATOS_ACC_VICT_20*/TABLA_PERS_*.csv
vehic=!ls ./cleaned/MICRODATOS_ACC_VICT_20*/TABLA_VEHIC_*.csv
dic_all_files={'Accvict':accvict,'Pers':pers,'Vehic':vehic}
len(dic_all_files)

3

#### RUTAS DE TODOS LOS FICHEROS ####

In [95]:
dic_all_files

{'Accvict': ['./cleaned/MICRODATOS_ACC_VICT_2008/TABLA_ACCVICT_24H_2008.csv',
  './cleaned/MICRODATOS_ACC_VICT_2009/TABLA_ACCVICT_24H_2009.csv',
  './cleaned/MICRODATOS_ACC_VICT_2010/TABLA_ACCVICT_24H_2010.csv',
  './cleaned/MICRODATOS_ACC_VICT_2011/TABLA_ACCVICT_2011.csv',
  './cleaned/MICRODATOS_ACC_VICT_2012/TABLA_ACCVICT_2012.csv',
  './cleaned/MICRODATOS_ACC_VICT_2013/TABLA_ACCVICT_2013.csv',
  './cleaned/MICRODATOS_ACC_VICT_2014/TABLA_ACCVICT_2014.csv',
  './cleaned/MICRODATOS_ACC_VICT_2015/TABLA_ACCVICT_2015.csv'],
 'Pers': ['./cleaned/MICRODATOS_ACC_VICT_2008/TABLA_PERS_24H_2008.csv',
  './cleaned/MICRODATOS_ACC_VICT_2009/TABLA_PERS_24H_2009.csv',
  './cleaned/MICRODATOS_ACC_VICT_2010/TABLA_PERS_24H_2010.csv',
  './cleaned/MICRODATOS_ACC_VICT_2011/TABLA_PERS_2011.csv',
  './cleaned/MICRODATOS_ACC_VICT_2012/TABLA_PERS_2012.csv',
  './cleaned/MICRODATOS_ACC_VICT_2013/TABLA_PERS_2013.csv',
  './cleaned/MICRODATOS_ACC_VICT_2014/TABLA_PERS_2014.csv',
  './cleaned/MICRODATOS_ACC_VICT

### Ahora que los datos estan trazados, procedemos a realizar una observacion preliminar para comprobar si hay homgeneidad en los datos, y si disponemos de la misma informacion en cada data set, ya que segun la DGT, las extructuras de los datos cambian a partir del 2011 en hasta el 2015.

### Por los que tenemos dos grupos de datos:

### Tipo 1) 2008-2010
### Tipo 2) 2011-2015

### Recorro cada clave - para obtener las cabeceras y poder comparar que campos tengo en cada año por tipo de fichero, e introduzco los data frames en una lista.

In [96]:
dic_all_files.keys()

dict_keys(['Accvict', 'Pers', 'Vehic'])

In [97]:
#Creo una lista con las cabeceras de los ficheros
Headers_List=[]
for key in dic_all_files.keys():
    df_2008=pd.read_csv(dic_all_files[key][0],delimiter=';',nrows=0,encoding="ISO-8859-1")
    df_2008=df_2008.transpose()
    df_2008.reset_index(inplace=True)
    df_2008.columns=[key+'_2008']

    df_2015=pd.read_csv(dic_all_files[key][7],delimiter=';',nrows=0,encoding="ISO-8859-1")
    df_2015=df_2015.transpose()
    df_2015.reset_index(inplace=True)
    df_2015.columns=[key+'_2015']
        
    Headers_List.append(df_2008.merge(df_2015, left_on=key+'_2008', right_on=key+'_2015', how='outer'))

### --------------------------------------------------------------------------------------------------

## REVISION DE LOS TRES TIPOS DE DATA SETS

### SE DISPONE DE TRES TIPOS DE DATASETS:

#### 1. ACCIDENTES
#### 2. PERSONAS
#### 3 .VEHICULOS

#### COMO PRIMER PASO VOY A REVISAR TANTO ESTRUCTURA, PARA DECIDIR QUE VARIABLES MANTENDRE PARA EL ANALISIS POSTERIOR. PROCEDERE A MODIFICAR LOS DATASET DE LA CARPETA "./CLEANED" ANTES DE REALIZAR LA CARGA PARA EN ANALISIS.

### --------------------------------------------------------------------------------------------------

###   1. ACCIDENTES

In [98]:
#Cabecera del fichero de Accidentes_victimas
Header_Accvict=Headers_List[0]
Header_Accvict

,Accvict_2008,Accvict_2015
0,ID_ACCIDENTE,ID_ACCIDENTE
1,ANIO,ANIO
2,MES,MES
3,HORA,HORA
4,DIASEMANA,DIASEMANA
5,PROVINCIA,PROVINCIA
6,COMUNIDAD_AUTONOMA,COMUNIDAD_AUTONOMA
7,ISLA,ISLA
8,MUNICIPIO,MUNICIPIO
9,TOT_VICTIMAS,TOT_VICTIMAS


In [99]:
#mantengo solo las columnas que son iguales en los dos tipos de dataset

Header_Accvict=Header_Accvict[Header_Accvict['Accvict_2008']==Header_Accvict['Accvict_2015']]
Header_Accvict=Header_Accvict['Accvict_2008'].tolist()
dic_all_headers={'Accvict':Header_Accvict}
dic_all_headers

#Genero nuevo DF vacio con los nombres de columnas
df_all_years=pd.DataFrame(index=None, columns=dic_all_headers['Accvict'])

#Primero genero una vista con todos los datos para examinarlos
for file in dic_all_files['Accvict']:
    df_vict=pd.read_csv(file,delimiter=';',usecols=dic_all_headers['Accvict'],encoding="ISO-8859-1",dtype={'ID_ACCIDENTE': object})
    #relleno los nulos con 999
    df_vict.fillna(999,inplace=True)
    df_all_years=pd.concat([df_all_years,df_vict],ignore_index=True)


#muestreo los datos dde la vista
df_all_years.head(3)
df_all_years.tail(3)
df_all_years.sample(3)

#Decido descartar los siguientes campos, al no considerarlos relevantes o estar muy incompletos:
df_all_years.drop(['ISLA','MUNICIPIO','TIPO_INTERSEC','ACERAS','CARRETERA','TRAZADO_NO_INTERSEC'], axis=1, inplace=True)

#Actualizo mi diccionario con las cabeceras que utilizare en el proceso de carga
Header_Accvict=df_all_years.columns.tolist()
dic_all_headers={'Accvict':Header_Accvict}
dic_all_headers
#En el campo factores atmosfericos, como lo considero relevante
#y tengo 4787 valores desconocidos, englobo estos registros dentro de la categoria otros(9)

#Vacio el df de la vista, que contendra todos los años
df_all_years=pd.DataFrame(index=None, columns=dic_all_headers['Accvict'])

for file in dic_all_files['Accvict']:
    df_acc=pd.read_csv(file,delimiter=";",usecols=dic_all_headers['Accvict'],encoding="ISO-8859-1",dtype={'ID_ACCIDENTE': object})
    df_acc.fillna(999,inplace=True)

    #factores atmosfericos desconocidos,tal y como cometo arriba van a otros
    df_acc.ix[(df_acc['FACTORES_ATMOSFERICOS']==999),'FACTORES_ATMOSFERICOS'] = 9
    #df_acc.ix[(df_vict['VISIBILIDAD_RESTRINGIDA']==999), 'VISIBILIDAD_RESTRINGIDA'] = df_all_years['VISIBILIDAD_RESTRINGIDA'][df_all_years['VISIBILIDAD_RESTRINGIDA']!=999].median()
    
    #Recorro todas las columnas del df, sustituyendo los known por la mediana del dato para tipos != objeto 'O'
    for col in df_acc.columns:
        if df_acc[col].dtype !='O':
            df_acc.ix[(df_acc[col]==999), col]=df_acc[col][df_acc[col]!=999].median()
            df_acc[col]=df_acc[col].astype(int)
        
    df_all_years=pd.concat([df_all_years,df_acc],ignore_index=True)
    df_acc.to_csv(file,index=False,sep=";",encoding="ISO-8859-1")
    print("MODFICADO ----- "+file)

#Convierto tambien los numericos de la vista tipo float a enteros
#Compruebo que me ha sustituido correctamente los 999 por la mediana.
for col in df_all_years.columns:
    if df_all_years[col].dtype !='O':
        df_all_years[col]=df_all_years[col].astype(int)
    df_all_years[col].value_counts(dropna=False)


print ("-----------------------------------------------------\n")
print ("VISTA CON LOS DATOS DE ACCIDENTES DESDE 2008 A 2015\n")
print ("-----------------------------------------------------\n")


# EL TENER CASI TODOS LOS DATASET CON VARIABLES CATEGORICAS, ME COMPLICA LA REALIZACION DE MODELOS.
# EN PRIMER LUGAR VOY A DECODIFICAR LAS COLUMNAS QUE CONSIDERO MAS RELEVANTES, ESTO ME DARA UN MEJOR VISION 

Decoder= {          
    
    "MES": {1:'Enero',2:'Febrero',3:'Marzo',4:'Abril',5:'Mayo',6:'Junio',7:'Julio',8:'Agosto',
                                 9:'Septiembre',10:'Octubre',11:'Noviembre',12:'Diciembre'},
    
    "DIASEMANA": {1:'LUNES',2:'MARTES',3:'MIÉRCOLES',4:'JUEVES',5:'VIERNES',6:'SÁBADO',7:'DOMINGO'},

    "PROVINCIA": {1:'Araba/Alava',2:'Albacete',3:'Alicante/Alacant',4:'Almería',5:'Avila',6:'Badajoz',
                     7:'Balears,_Illes',8:'Barcelona',9:'Burgos',10:'CAceres',11:'CAdiz',
                     12:'Castellon/Castello',13:'Ciudad_Real',14:'Cordoba',15:'Coruña,_A',16:'Cuenca',
                     17:'Girona',18:'Granada',19:'Guadalajara',20:'Gipuzkoa',21:'Huelva',22:'Huesca',
                     23:'Jaen',24:'Leon',25:'Lleida',26:'Rioja,_La',27:'Lugo',28:'Madrid',29:'Malaga',
                     30:'Murcia',31:'Navarra',32:'Ourense',33:'Asturias',34:'Palencia',35:'Palmas,_Las',
                     36:'Pontevedra',37:'Salamanca',38:'S.C.Tenerife',39:'Cantabria',40:'Segovia',
                     41:'Sevilla',42:'Soria',43:'Tarragona',44:'Teruel',45:'Toledo',46:'Valencia/Valencia',
                                 47:'Valladolid',48:'Bizkaia',49:'Zamora',50:'Zaragoza',51:'Ceuta',52:'Melilla'},
    
    "COMUNIDAD_AUTONOMA": {1:'Andalucia',2:'Aragon',3:'Asturias',4:'Baleares',5:'Canarias',6:'Cantabria',
            7:'Castilla_Leon',8:'Castilla_Mancha',9:'Cataluña',10:'Valencia',11:'Extremadura',
            12:'Galicia',13:'Madrid',14:'Murcia',15:'Navarra',16:'Rioja',17:'Pais_vasco',18:'Ceuta_Melilla'},
          
    "ZONA": {1:'CARRETERA',2:'ZONA URBANA',3:'TRAVESIA',4:'VARIANTE'},
          
    "ZONA_AGRUPADA": {1:'VÍAS INTERURBANAS',2:'VÍAS URBANAS'},
          
    "RED_CARRETERA": {1:'TITULARIDAD_ESTATAL',2:'TITULARIDAD_AUTONOMICA',3:'TITULARIDAD_PROVINCIAL',
                      4:'TITULARIDAD_MUNICIPAL',5:'OTRAS_TITULARIDADES'},
         
    "TIPO_VIA": {1:'AUTOPISTA',2:'AUTOVIA',3:'PARA_AUTOMOVILES',4:'CONVENCIONAL_CARRIL_LENTO',5:'CONVENCIONAL',
                 6:'CAMINO_VECINAL',7:'DE_SERVICIO',8:'RAMAL_DE_ENLACE',9:'OTRA'},

    "SUPERFICIE_CALZADA": {1:'SECA_Y_LIMPIA',2:'UMBRIA',3:'MOJADA',4:'HELADA',5:'NEVADA',6:'BARRILLO',
                      7:'GRAVILLA SUELTA',8:'ACEITE',9:'OTRO TIPO'},

    "LUMINOSIDAD": {1:'PLENO_DIA',2:'CREPUSCULO',3:'NOCHE_ILUMINACION_SUFICIENTE',4:'NOCHE_ILUMINACION_INSUFICIENTE',
                      5:'NOCHE: SIN ILUMINACION'},

    "FACTORES_ATMOSFERICOS": {1:'BUEN_TIEMPO',2:'NIEBLA_INTENSA',3:'NIEBLA_LIGERA',4:'LLOVIZNANDO',5:'LLUVIA_FUERTE',
                      6:'GRANIZANDO',7:'NEVANDO',8:'VIENTO FUERTE',9:'OTRO'},

    "TIPO_ACCIDENTE": {11:'Colision_de_vehiculos_en_marcha_(Frontal)',12:'Colision_de_vehiculos_en_marcha_(Frontolateral)',
                      13:'Colision_de_vehiculos_en_marcha_(Lateral)',14:'Colision_de_vehiculos_en_marcha_(Alcance)',
                      15:'Colision_de_vehiculos_en_marcha_(Multiple_o_en_caravana)',
                      21:'Colision_de_vehiculo_con_obstaculo_en_calzada_(Vehiculo_estacionado_o_averiado)',
                      22:'Colision_de_vehiculo_con_obstaculo_en_calzada_(Valla_de_defensa)',
                      23:'Colision_de_vehiculo_con_obstaculo_en_calzada_(Barrera_de_paso_a_nivel)',
                      24:'Colision_de_vehiculo_con_obstaculo_en_calzada_(Otro_objeto_o_material)',
                      31:'Atropello_a_peaton_sosteniendo_bicicleta',32:'Atropello_a_peaton_reparando_vehiculo',
                      33:'Atropello_a_peaton_aislado_o_en_grupo',34:'Atropello_a_conductor_de_animales',
                      35:'Atropello_a_animal_conducido_o_en_rebaño',
                      36:'Atropello_a_animales_sueltos',41:'Vuelco_en_la_calzada',
                      51:'Salida_de_la_via_por_la_izquierda_con_colision_(Choque_con_arbol_o_poste)',
                      52:'Salida_de_la_via_por_la_izquierda_con_colision_(Choque_con_muro_o_edificio)',
                      53:'Salida_de_la_via_por_la_izquierda_con_colision_(Choque_con_cuneta_o_bordillo)',
                      54:'Salida_de_la_via_por_la_izquierda_con_colision_(Otro_tipo_de_choque)',
                      55:'Salida_de_la_via_por_la_izquierda_sin_colision_(Con_despeñamiento)',
                      56:'Salida_de_la_via_por_la_izquierda_sin_colision_(Con_vuelco)',
                      57:'Salida_de_la_via_por_la_izquierda_sin_colision_(En_llano)',
                      58:'Salida_de_la_via_por_la_izquierda_sin_colision_(Otra)',
                      61:'Salida_de_la_via_por_la_derecha_con_colision_(Choque_con_arbol_o_poste)',
                      62:'Salida_de_la_via_por_la_derecha_con_colision_(Choque_con_muro_o_edificio)',
                      63:'Salida_de_la_via_por_la_derecha_con_colision_(Choque_con_cuneta_o_bordillo)',
                      64:'Salida_de_la_via_por_la_derecha_con_colision_(Otro_tipo_de_choque)',
                      65:'Salida_de_la_via_por_la_derecha_sin_colision_(Con_despeñamiento)',
                      66:'Salida_de_la_via_por_la_derecha_sin_colision_(Con_vuelco)',
                      67:'Salida_de_la_via_por_la_derecha_sin_colision_(En_llano)',
                      68:'Salida_de_la_via_por_la_derecha_sin_colision_(Otra)',71:'Otro_tipo_de_accidente'}
         }

df_all_years.replace(Decoder, inplace=True)

df_all_years.to_csv('./cleaned/view_accvict_08_15.csv',index=False,sep=";",encoding="ISO-8859-1")
print("CREADA VISTA CON DATOS DE ACCIDENTES Y VISTIMAS DE 2008 A 2015")

df_all_years=pd.read_csv('./cleaned/view_accvict_08_15.csv',delimiter=";",encoding="ISO-8859-1",dtype={'ID_ACCIDENTE': object})
df_all_years.info()
df_all_years.head()
df_all_years.tail()

########   FIN DEL PROCESAMIENTO DE LOS DATASET DE ACCIDENTES #########



{'Accvict': ['ID_ACCIDENTE',
  'ANIO',
  'MES',
  'HORA',
  'DIASEMANA',
  'PROVINCIA',
  'COMUNIDAD_AUTONOMA',
  'ISLA',
  'MUNICIPIO',
  'TOT_VICTIMAS',
  'TOT_MUERTOS',
  'TOT_HERIDOS_GRAVES',
  'TOT_HERIDOS_LEVES',
  'TOT_VEHICULOS_IMPLICADOS',
  'ZONA',
  'ZONA_AGRUPADA',
  'CARRETERA',
  'RED_CARRETERA',
  'TIPO_VIA',
  'TRAZADO_NO_INTERSEC',
  'TIPO_INTERSEC',
  'SUPERFICIE_CALZADA',
  'LUMINOSIDAD',
  'FACTORES_ATMOSFERICOS',
  'VISIBILIDAD_RESTRINGIDA',
  'ACERAS',
  'TIPO_ACCIDENTE']}

,ID_ACCIDENTE,ANIO,MES,HORA,DIASEMANA,PROVINCIA,COMUNIDAD_AUTONOMA,ISLA,MUNICIPIO,TOT_VICTIMAS,TOT_MUERTOS,TOT_HERIDOS_GRAVES,TOT_HERIDOS_LEVES,TOT_VEHICULOS_IMPLICADOS,ZONA,ZONA_AGRUPADA,CARRETERA,RED_CARRETERA,TIPO_VIA,TRAZADO_NO_INTERSEC,TIPO_INTERSEC,SUPERFICIE_CALZADA,LUMINOSIDAD,FACTORES_ATMOSFERICOS,VISIBILIDAD_RESTRINGIDA,ACERAS,TIPO_ACCIDENTE
0,2008480000372,2008.0,4.0,7.0,5.0,48.0,17.0,999.0,999.0,4.0,0.0,0.0,4.0,1.0,1.0,1.0,BI-631,3.0,2.0,1.0,999.0,2.0,3.0,1.0,8.0,N,71.0
1,2008250021218,2008.0,3.0,14.0,7.0,25.0,9.0,999.0,999.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,C-12,2.0,5.0,2.0,999.0,1.0,1.0,9.0,8.0,N,22.0
2,2008430024095,2008.0,10.0,12.0,7.0,43.0,9.0,999.0,999.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,TV-7331,4.0,5.0,1.0,999.0,1.0,1.0,1.0,0.0,N,68.0


,ID_ACCIDENTE,ANIO,MES,HORA,DIASEMANA,PROVINCIA,COMUNIDAD_AUTONOMA,ISLA,MUNICIPIO,TOT_VICTIMAS,TOT_MUERTOS,TOT_HERIDOS_GRAVES,TOT_HERIDOS_LEVES,TOT_VEHICULOS_IMPLICADOS,ZONA,ZONA_AGRUPADA,CARRETERA,RED_CARRETERA,TIPO_VIA,TRAZADO_NO_INTERSEC,TIPO_INTERSEC,SUPERFICIE_CALZADA,LUMINOSIDAD,FACTORES_ATMOSFERICOS,VISIBILIDAD_RESTRINGIDA,ACERAS,TIPO_ACCIDENTE
711899,2016990000875,2015.0,12.0,11.0,4.0,28.0,13.0,999.0,28079.0,2.0,0.0,0.0,2.0,2.0,2.0,2.0,999,4.0,8.0,999.0,2.0,3.0,1.0,4.0,999.0,998,2.0
711900,2016990000876,2015.0,12.0,13.0,2.0,28.0,13.0,999.0,28079.0,1.0,0.0,0.0,1.0,1.0,2.0,2.0,999,4.0,8.0,1.0,999.0,1.0,1.0,1.0,999.0,999,7.0
711901,2016990001660,2015.0,11.0,15.0,1.0,28.0,13.0,999.0,28079.0,1.0,0.0,0.0,1.0,2.0,2.0,2.0,999,4.0,8.0,999.0,2.0,1.0,1.0,1.0,999.0,998,2.0


,ID_ACCIDENTE,ANIO,MES,HORA,DIASEMANA,PROVINCIA,COMUNIDAD_AUTONOMA,ISLA,MUNICIPIO,TOT_VICTIMAS,TOT_MUERTOS,TOT_HERIDOS_GRAVES,TOT_HERIDOS_LEVES,TOT_VEHICULOS_IMPLICADOS,ZONA,ZONA_AGRUPADA,CARRETERA,RED_CARRETERA,TIPO_VIA,TRAZADO_NO_INTERSEC,TIPO_INTERSEC,SUPERFICIE_CALZADA,LUMINOSIDAD,FACTORES_ATMOSFERICOS,VISIBILIDAD_RESTRINGIDA,ACERAS,TIPO_ACCIDENTE
288721,2011080014841,2011.0,11.0,19.0,2.0,8.0,9.0,999.0,8019.0,2.0,0.0,0.0,2.0,2.0,2.0,2.0,999,4.0,8.0,999.0,2.0,1.0,3.0,1.0,999.0,998,20.0
530602,201407040001129,2014.0,8.0,10.0,2.0,7.0,4.0,71.0,7040.0,1.0,0.0,0.0,1.0,1.0,2.0,2.0,999,4.0,8.0,999.0,999.0,1.0,1.0,1.0,999.0,0,7.0
296365,2011140000811,2011.0,6.0,8.0,5.0,14.0,1.0,999.0,14021.0,1.0,0.0,0.0,1.0,1.0,2.0,2.0,999,4.0,8.0,1.0,999.0,1.0,2.0,1.0,1.0,998,9.0


{'Accvict': ['ID_ACCIDENTE',
  'ANIO',
  'MES',
  'HORA',
  'DIASEMANA',
  'PROVINCIA',
  'COMUNIDAD_AUTONOMA',
  'TOT_VICTIMAS',
  'TOT_MUERTOS',
  'TOT_HERIDOS_GRAVES',
  'TOT_HERIDOS_LEVES',
  'TOT_VEHICULOS_IMPLICADOS',
  'ZONA',
  'ZONA_AGRUPADA',
  'RED_CARRETERA',
  'TIPO_VIA',
  'SUPERFICIE_CALZADA',
  'LUMINOSIDAD',
  'FACTORES_ATMOSFERICOS',
  'VISIBILIDAD_RESTRINGIDA',
  'TIPO_ACCIDENTE']}

...

1    421417
8    151022
0     94613
2     11739
7     11337
3     10176
4      7755
5      2681
6      1162
Name: VISIBILIDAD_RESTRINGIDA, dtype: int64

2     89924
4     82990
12    70554
7     55923
14    51057
3     34237
33    29470
20    27167
5     24207
13    22316
19    21280
16    21037
15    19466
71    15121
9     14116
1     13998
6     12085
68    10606
18    10454
11    10194
41     8905
10     7449
66     7166
64     5987
24     5595
56     4428
54     4363
63     4289
62     3068
52     2917
53     2890
61     2886
8      2387
67     2095
21     1853
51     1760
17     1732
65     1170
36     1091
57     1013
22      855
55      798
58      536
31      256
32       94
35       55
23       37
34       25
Name: TIPO_ACCIDENTE, dtype: int64

-----------------------------------------------------

VISTA CON LOS DATOS DE ACCIDENTES DESDE 2008 A 2015

-----------------------------------------------------

CREADA VISTA CON DATOS DE ACCIDENTES Y VISTIMAS DE 2008 A 2015
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711902 entries, 0 to 711901
Data columns (total 21 columns):
ID_ACCIDENTE                711902 non-null object
ANIO                        711902 non-null int64
MES                         711902 non-null object
HORA                        711902 non-null int64
DIASEMANA                   711902 non-null object
PROVINCIA                   711902 non-null object
COMUNIDAD_AUTONOMA          711902 non-null object
TOT_VICTIMAS                711902 non-null int64
TOT_MUERTOS                 711902 non-null int64
TOT_HERIDOS_GRAVES          711902 non-null int64
TOT_HERIDOS_LEVES           711902 non-null int64
TOT_VEHICULOS_IMPLICADOS    711902 non-null int64
ZONA                        711902 non-null object
ZONA_AG

,ID_ACCIDENTE,ANIO,MES,HORA,DIASEMANA,PROVINCIA,COMUNIDAD_AUTONOMA,TOT_VICTIMAS,TOT_MUERTOS,TOT_HERIDOS_GRAVES,TOT_HERIDOS_LEVES,TOT_VEHICULOS_IMPLICADOS,ZONA,ZONA_AGRUPADA,RED_CARRETERA,TIPO_VIA,SUPERFICIE_CALZADA,LUMINOSIDAD,FACTORES_ATMOSFERICOS,VISIBILIDAD_RESTRINGIDA,TIPO_ACCIDENTE
0,2008480000372,2008,Abril,7,VIERNES,Bizkaia,Pais_vasco,4,0,0,4,1,CARRETERA,VÍAS INTERURBANAS,TITULARIDAD_PROVINCIAL,AUTOVIA,UMBRIA,NOCHE_ILUMINACION_SUFICIENTE,BUEN_TIEMPO,8,Otro_tipo_de_accidente
1,2008250021218,2008,Marzo,14,DOMINGO,Lleida,Cataluña,1,0,1,0,1,CARRETERA,VÍAS INTERURBANAS,TITULARIDAD_AUTONOMICA,CONVENCIONAL,SECA_Y_LIMPIA,PLENO_DIA,OTRO,8,Colision_de_vehiculo_con_obstaculo_en_calzada_...
2,2008430024095,2008,Octubre,12,DOMINGO,Tarragona,Cataluña,1,0,0,1,1,CARRETERA,VÍAS INTERURBANAS,TITULARIDAD_MUNICIPAL,CONVENCIONAL,SECA_Y_LIMPIA,PLENO_DIA,BUEN_TIEMPO,0,Salida_de_la_via_por_la_derecha_sin_colision_(...
3,2008430022973,2008,Abril,18,MARTES,Tarragona,Cataluña,1,0,0,1,2,CARRETERA,VÍAS INTERURBANAS,TITULARIDAD_MUNICIPAL,CONVENCIONAL,SECA_Y_LIMPIA,PLENO_DIA,BUEN_TIEMPO,0,Colision_de_vehiculos_en_marcha_(Alcance)
4,2008250021457,2008,Mayo,19,VIERNES,Lleida,Cataluña,1,0,1,0,1,CARRETERA,VÍAS INTERURBANAS,TITULARIDAD_MUNICIPAL,OTRA,OTRO TIPO,PLENO_DIA,OTRO,8,Salida_de_la_via_por_la_derecha_sin_colision_(...


,ID_ACCIDENTE,ANIO,MES,HORA,DIASEMANA,PROVINCIA,COMUNIDAD_AUTONOMA,TOT_VICTIMAS,TOT_MUERTOS,TOT_HERIDOS_GRAVES,TOT_HERIDOS_LEVES,TOT_VEHICULOS_IMPLICADOS,ZONA,ZONA_AGRUPADA,RED_CARRETERA,TIPO_VIA,SUPERFICIE_CALZADA,LUMINOSIDAD,FACTORES_ATMOSFERICOS,VISIBILIDAD_RESTRINGIDA,TIPO_ACCIDENTE
711897,2016990000448,2015,Diciembre,13,MARTES,Madrid,Madrid,1,0,0,1,1,ZONA URBANA,VÍAS URBANAS,TITULARIDAD_MUNICIPAL,RAMAL_DE_ENLACE,SECA_Y_LIMPIA,PLENO_DIA,BUEN_TIEMPO,1,7
711898,2016990000873,2015,Diciembre,7,JUEVES,Madrid,Madrid,1,0,0,1,2,ZONA URBANA,VÍAS URBANAS,TITULARIDAD_MUNICIPAL,RAMAL_DE_ENLACE,MOJADA,NOCHE_ILUMINACION_SUFICIENTE,LLOVIZNANDO,1,2
711899,2016990000875,2015,Diciembre,11,JUEVES,Madrid,Madrid,2,0,0,2,2,ZONA URBANA,VÍAS URBANAS,TITULARIDAD_MUNICIPAL,RAMAL_DE_ENLACE,MOJADA,PLENO_DIA,LLOVIZNANDO,1,2
711900,2016990000876,2015,Diciembre,13,MARTES,Madrid,Madrid,1,0,0,1,1,ZONA URBANA,VÍAS URBANAS,TITULARIDAD_MUNICIPAL,RAMAL_DE_ENLACE,SECA_Y_LIMPIA,PLENO_DIA,BUEN_TIEMPO,1,7
711901,2016990001660,2015,Noviembre,15,LUNES,Madrid,Madrid,1,0,0,1,2,ZONA URBANA,VÍAS URBANAS,TITULARIDAD_MUNICIPAL,RAMAL_DE_ENLACE,SECA_Y_LIMPIA,PLENO_DIA,BUEN_TIEMPO,1,2


In [100]:
dic_all_files["Accvict"]


['./cleaned/MICRODATOS_ACC_VICT_2008/TABLA_ACCVICT_24H_2008.csv',
 './cleaned/MICRODATOS_ACC_VICT_2009/TABLA_ACCVICT_24H_2009.csv',
 './cleaned/MICRODATOS_ACC_VICT_2010/TABLA_ACCVICT_24H_2010.csv',
 './cleaned/MICRODATOS_ACC_VICT_2011/TABLA_ACCVICT_2011.csv',
 './cleaned/MICRODATOS_ACC_VICT_2012/TABLA_ACCVICT_2012.csv',
 './cleaned/MICRODATOS_ACC_VICT_2013/TABLA_ACCVICT_2013.csv',
 './cleaned/MICRODATOS_ACC_VICT_2014/TABLA_ACCVICT_2014.csv',
 './cleaned/MICRODATOS_ACC_VICT_2015/TABLA_ACCVICT_2015.csv']

###   2. VEHICULOS

In [101]:
#Cabecera del fichero de Vehiculos
Header_Vehic=Headers_List[2]
Header_Vehic

,Vehic_2008,Vehic_2015
0,ID_ACCIDENTE,ID_ACCIDENTE
1,ID_VEHICULO,ID_VEHICULO
2,ANIO_MATRICULA_VEHICULO,ANIO_MATRICULA_VEHICULO
3,MES_MATRICULA_VEHICULO,MES_MATRICULA_VEHICULO
4,TIPO_VEHICULO,TIPO_VEHICULO
5,ESTADO_VEHICULO,NaN
6,NUMERO_OCUPANTES_VEH,NUMERO_OCUPANTES_VEH
7,MERCANCIAS_PELIGROSAS,MERCANCIAS_PELIGROSAS
8,VEHICULO_INCENDIADO,VEHICULO_INCENDIADO
9,ANIO,ANIO


In [103]:
#Matego solo las columnas inguales y actualizo mi diccionario con las cabeceras que utilizare en el proceso de carga

Header_Vehic=Header_Vehic[Header_Vehic['Vehic_2008']==Header_Vehic['Vehic_2015']]
Header_Vehic=Header_Vehic['Vehic_2008'].tolist()
dic_all_headers={'Vehic':Header_Vehic}
dic_all_headers

#Genero un df vacio donde concatenare todos los años para ver la cantidad de valores de que 
#dispongo por cada columna

df_all_years=pd.DataFrame(index=None, columns=dic_all_headers['Vehic'])

#Cargo cada csv y concateno
for file in dic_all_files['Vehic']:
    df_vehic=pd.read_csv(file,delimiter=';',usecols=dic_all_headers['Vehic'],encoding="ISO-8859-1",dtype={'ID_ACCIDENTE': object})
    #relleno los nulos con 999
    df_vehic.fillna(999,inplace=True)
    df_all_years=pd.concat([df_all_years,df_vehic],ignore_index=True)
    
#muestreo los datos
df_all_years.head(3)
df_all_years.tail(3)
df_all_years.sample(3)

#De las columnas del data frame final, que contiene todos los años, 
#examino los distintos valores de las columnas, para tratar de decidir 
#que campos conservare para el analisis.


#Una vez examindas los resultado anteriores, decido descartar los siguientes campos de la vista:
df_all_years.drop(['MERCANCIAS_PELIGROSAS','VEHICULO_INCENDIADO'], axis=1, inplace=True)

#Actualizo mi diccionario con las cabeceras que utilizare en el proceso de carga
Header_Vehic=df_all_years.columns.tolist()
dic_all_headers={'Vehic':Header_Vehic}
dic_all_headers
#------------------SOBREECRITURA DE LOS DATOS APLICANDO MODIFICACIONES--------------
#Decido eliminor las siguientes columnas, por el alto numero de registro desconocidos y 
#porque considero que no me aportan valor:

#MERCANCIAS_PELIGROSAS
#VEHICULO_INCENDIADO

#Vacio el df de la vista, que contendra todos los años
df_all_years=pd.DataFrame(index=None, columns=dic_all_headers['Vehic'])

for file in dic_all_files['Vehic']:
    df_vehic=pd.read_csv(file,delimiter=';',usecols=dic_all_headers['Vehic'],encoding="ISO-8859-1",dtype={'ID_ACCIDENTE': object})
    #relleno los nulos con 999
    df_vehic.fillna(999,inplace=True)
    for col in df_vehic.columns:
        if df_vehic[col].dtype !='O':
            df_vehic.ix[(df_vehic[col]==999), col]=df_vehic[col][df_vehic[col]!=999].median()


    df_vehic.to_csv(file,index=False,sep=";",encoding="ISO-8859-1")
    df_all_years=pd.concat([df_all_years,df_vehic],ignore_index=True)
    print("MODFICADO ----- "+file)


#Convierto tambien los numericos de la vista tipo float a enteros
#Compruebo que me ha sustituido correctamente los 999 por la mediana.
for col in df_all_years.columns:
    if df_all_years[col].dtype !='O':
        df_all_years[col]=df_all_years[col].astype(int)
    df_all_years[col].value_counts(dropna=False)

# EL TENER CASI TODOS LOS DATASET CON VARIABLES CATEGORICAS, ME COMPLICA LA REALIZACION DE MODELOS.
# EN PRIMER LUGAR VOY A DECODIFICAR LAS COLUMNAS QUE CONSIDERO MAS RELEVANTES, ESTO ME DARA UN MEJOR VISION 

Decoder= { "TIPO_VEHICULO": 
          { 1:'Bicicleta_o_triciclo_sin_motor',
            2:'Ciclomotor',10:'Coche_de_Minusvalido',11:'Motocicleta',
            21:'Turismo_de_SP_hasta_9_plazas',22:'Turismo_sin_remolque',23:'Turismo_con_remolque',
            24:'Ambulancia',30:'Maquinaria_de_obras_y_agricola',31:'Tractor_agricola_sin_remolque',
            32:'Tractor_agricola_con_remolque',41:'Camion(PM<=3500_K)_sin_remolque',
            42:'Camion(PM<=3500_K)_con_remolque',43:'Furgoneta',51:'Camion(PM>3500_K)_sin_remolque',
            52:'Camion(PM>3500_K)_con_remolque',53:'Camion_cisterna_sin_remolque',
            54:'Camion_cisterna_con_remolque',55:'Vehiculo_articulado',61:'Autobus_de_linea_regular',
            62:'Autobus_escolar',63:'Otro_autobus',70:'Tren',80:'Carro',81:'Otros_Vehiculos',
            82:'Cuadriciclo',4:'DESCONOCIDO',6:'DESCONOCIDO',9:'DESCONOCIDO',3:'DESCONOCIDO',
            5:'DESCONOCIDO',16:'DESCONOCIDO',12:'DESCONOCIDO',15:'DESCONOCIDO',20:'DESCONOCIDO',
            13:'DESCONOCIDO',8:'DESCONOCIDO',90:'DESCONOCIDO',7:'DESCONOCIDO',18:'DESCONOCIDO',
            14:'DESCONOCIDO',19:'DESCONOCIDO',17:'DESCONOCIDO'}
         }

df_all_years.replace(Decoder, inplace=True)
df_all_years.head()
df_all_years.tail()
            
print("CREADA VISTA CON DATOS DE VEHICULOS DE 2008 A 2015")
df_all_years.to_csv("./cleaned/view_vehic_08_15.csv",index=False,sep=";",encoding="ISO-8859-1")
print("CREADA VISTA CON DATOS DE VEHICULOS DE 2008 A 2015")
df_all_years=pd.read_csv("./cleaned/view_vehic_08_15.csv",delimiter=";",encoding="ISO-8859-1",dtype={'ID_ACCIDENTE': object})
#comprobacion
df_all_years.info()
df_all_years.head()
df_all_years.tail()

########   FIN DEL PROCESAMIENTO DE LOS DATASET DE VEHICULOS #########


{'Vehic': ['ID_ACCIDENTE',
  'ID_VEHICULO',
  'ANIO_MATRICULA_VEHICULO',
  'MES_MATRICULA_VEHICULO',
  'TIPO_VEHICULO',
  'NUMERO_OCUPANTES_VEH',
  'MERCANCIAS_PELIGROSAS',
  'VEHICULO_INCENDIADO',
  'ANIO']}

,ID_ACCIDENTE,ID_VEHICULO,ANIO_MATRICULA_VEHICULO,MES_MATRICULA_VEHICULO,TIPO_VEHICULO,NUMERO_OCUPANTES_VEH,MERCANCIAS_PELIGROSAS,VEHICULO_INCENDIADO,ANIO
0,2008410002330,2.0,9999.0,99.0,22.0,1.0,N,N,2008.0
1,2008520000162,2.0,1991.0,99.0,22.0,1.0,N,N,2008.0
2,2009460000665,3.0,2004.0,1.0,22.0,1.0,N,N,2008.0


,ID_ACCIDENTE,ID_VEHICULO,ANIO_MATRICULA_VEHICULO,MES_MATRICULA_VEHICULO,TIPO_VEHICULO,NUMERO_OCUPANTES_VEH,MERCANCIAS_PELIGROSAS,VEHICULO_INCENDIADO,ANIO
1239590,2016990000876,1.0,2011.0,1.0,6.0,1.0,0,999,2015.0
1239591,2016990001660,1.0,9999.0,99.0,6.0,1.0,0,999,2015.0
1239592,2016990001660,2.0,2011.0,1.0,6.0,4.0,0,999,2015.0


,ID_ACCIDENTE,ID_VEHICULO,ANIO_MATRICULA_VEHICULO,MES_MATRICULA_VEHICULO,TIPO_VEHICULO,NUMERO_OCUPANTES_VEH,MERCANCIAS_PELIGROSAS,VEHICULO_INCENDIADO,ANIO
969563,201414042000037,1.0,2001.0,10.0,6.0,1.0,0,999,2014.0
123012,2008990021452,2.0,2005.0,99.0,22.0,999.0,999,999,2008.0
383687,2011060000035,1.0,2005.0,7.0,22.0,1.0,N,N,2010.0


{'Vehic': ['ID_ACCIDENTE',
  'ID_VEHICULO',
  'ANIO_MATRICULA_VEHICULO',
  'MES_MATRICULA_VEHICULO',
  'TIPO_VEHICULO',
  'NUMERO_OCUPANTES_VEH',
  'ANIO']}

...

,ID_ACCIDENTE,ID_VEHICULO,ANIO_MATRICULA_VEHICULO,MES_MATRICULA_VEHICULO,TIPO_VEHICULO,NUMERO_OCUPANTES_VEH,ANIO
0,2008410002330,2,9999,99,Turismo_sin_remolque,1,2008
1,2008520000162,2,1991,99,Turismo_sin_remolque,1,2008
2,2009460000665,3,2004,1,Turismo_sin_remolque,1,2008
3,2008460007725,1,2003,1,Turismo_sin_remolque,5,2008
4,2008410000371,1,2006,7,Turismo_sin_remolque,1,2008


,ID_ACCIDENTE,ID_VEHICULO,ANIO_MATRICULA_VEHICULO,MES_MATRICULA_VEHICULO,TIPO_VEHICULO,NUMERO_OCUPANTES_VEH,ANIO
1239588,2016990000875,1,2008,1,DESCONOCIDO,1,2015
1239589,2016990000875,2,2006,1,DESCONOCIDO,2,2015
1239590,2016990000876,1,2011,1,DESCONOCIDO,1,2015
1239591,2016990001660,1,9999,99,DESCONOCIDO,1,2015
1239592,2016990001660,2,2011,1,DESCONOCIDO,4,2015


CREADA VISTA CON DATOS DE VEHICULOS DE 2008 A 2015
CREADA VISTA CON DATOS DE VEHICULOS DE 2008 A 2015
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1239593 entries, 0 to 1239592
Data columns (total 7 columns):
ID_ACCIDENTE               1239593 non-null object
ID_VEHICULO                1239593 non-null int64
ANIO_MATRICULA_VEHICULO    1239593 non-null int64
MES_MATRICULA_VEHICULO     1239593 non-null int64
TIPO_VEHICULO              1239593 non-null object
NUMERO_OCUPANTES_VEH       1239593 non-null int64
ANIO                       1239593 non-null int64
dtypes: int64(5), object(2)
memory usage: 66.2+ MB


,ID_ACCIDENTE,ID_VEHICULO,ANIO_MATRICULA_VEHICULO,MES_MATRICULA_VEHICULO,TIPO_VEHICULO,NUMERO_OCUPANTES_VEH,ANIO
0,2008410002330,2,9999,99,Turismo_sin_remolque,1,2008
1,2008520000162,2,1991,99,Turismo_sin_remolque,1,2008
2,2009460000665,3,2004,1,Turismo_sin_remolque,1,2008
3,2008460007725,1,2003,1,Turismo_sin_remolque,5,2008
4,2008410000371,1,2006,7,Turismo_sin_remolque,1,2008


,ID_ACCIDENTE,ID_VEHICULO,ANIO_MATRICULA_VEHICULO,MES_MATRICULA_VEHICULO,TIPO_VEHICULO,NUMERO_OCUPANTES_VEH,ANIO
1239588,2016990000875,1,2008,1,DESCONOCIDO,1,2015
1239589,2016990000875,2,2006,1,DESCONOCIDO,2,2015
1239590,2016990000876,1,2011,1,DESCONOCIDO,1,2015
1239591,2016990001660,1,9999,99,DESCONOCIDO,1,2015
1239592,2016990001660,2,2011,1,DESCONOCIDO,4,2015


###   1. PERSONAS

### TRATAMIENTO DATA SET PERSONAS

### Repito el procedimiento anterior para los datos desde 2011 a 2015
### grabando los xls modificados en la carpeta cleaned junto con los otros data set 
### de años 2008 a 2010

In [88]:
#Cabecera del fichero de Personas
Header_Pers=Headers_List[1]
Header_Pers


,Pers_2008,Pers_2015
0,ID_ACCIDENTE,ID_ACCIDENTE
1,ID_VEHICULO,ID_VEHICULO
2,ID_PERSONA,ID_PERSONA
3,ID_CONDUCTOR,ID_CONDUCTOR
4,ID_PASAJERO,ID_PASAJERO
5,ID_PEATON,ID_PEATON
6,EDAD,EDAD
7,COD_SEXO,COD_SEXO
8,ANIO_PERMISO,ANIO_PERMISO
9,COD_POSICION,NaN


In [89]:
Header_Pers=Header_Pers[Header_Pers['Pers_2008']==Header_Pers['Pers_2015']]
Header_Pers=Header_Pers['Pers_2008'].tolist()
dic_all_headers={'Pers':Header_Pers}
dic_all_headers

#Genero la vista con los datos de todos los años
df_all_years=pd.DataFrame(index=None, columns=dic_all_headers['Pers'])

#Cargo de nuevo ya que he modificado las tablas que componen la vista y la genero de nuevo
#sustituyo en todos los años los valores 999 por la mediana
for file in dic_all_files['Pers']:
    df_pers=pd.read_csv(file,delimiter=';',usecols=dic_all_headers['Pers'],encoding="ISO-8859-1",dtype={'ID_ACCIDENTE': object})
    #relleno los nulos con 999
    df_pers.fillna(999,inplace=True)
    #Modifico los valores desconocidos para todos los ficheros, excepto para la columna año de permiso
    for col in df_pers.columns:
        if df_pers[col].dtype !='O':
            #convierto la columna a entero
            df_pers[col]=df_pers[col].astype(int)
            #Sustituyo los valores de años de permisos deconocidos
            if col == 'ANIO_PERMISO':
                    df_pers.ix[((df_pers[col]==9999) | (df_pers[col]==0.0)) & (df_pers['ID_PERSONA']==1), col]=df_pers[col][(df_pers['ID_PERSONA']==1) & (df_pers[col]!=9999)].median()
            else:
                df_pers.ix[(df_pers[col]==999), col]=df_pers[col][df_pers[col]!=999].median()
            
    #Acumulo los datos en un dataframe con todos los años       
    df_all_years=pd.concat([df_all_years,df_pers],ignore_index=True)
    #Grabo el fichero fuente con las modificaciones
    df_pers.to_csv(file,index=False,sep=";",encoding="ISO-8859-1")


#Convierto tambien los numericos de la vista tipo float a enteros
#Compruebo que me ha sustituido correctamente los 999 por la mediana.
for col in df_all_years.columns:
    if df_all_years[col].dtype !='O':
        df_all_years[col]=df_all_years[col].astype(int)
        df_all_years[col].value_counts(dropna=False)


# EL TENER CASI TODOS LOS DATASET CON VARIABLES CATEGORICAS, ME COMPLICA LA REALIZACION DE MODELOS.
# EN PRIMER LUGAR VOY A DECODIFICAR LAS COLUMNAS QUE CONSIDERO MAS RELEVANTES, ESTO ME DARA UN MEJOR VISION 

Decoder= {          
    
    "ID_PERSONA": {1:'CONDUCTOR',2:'PASAJERO',3:'PEATON'},

    "COD_SEXO": {'M':'MUJER','V':'HOMBRE','X':'SIN ESPECIFICAR'},

    "POSICION": {1:'CONDUCTOR VEHICULO',2:'PASAJERO DELANTERO',
                  3:'PASAJERO TRASERO IZQUIERDO',4:'PASAJERO TRASERO DERECHO',5:'PASAJERO TRASERO CENTRAL',
                  6:'CONDUCTOR VEHICULO DE DOS RUEDAS',7:'PASAJERO VEHICULO DE DOS RUEDAS',
                  8:'OTROS PASAJEROS SENTADOS',
                  9:'OTROS PASAJEROS DE PIE'},

    "MANIOBRAS": {0:'SIN DATO',1:'SIGUIENDO LA RUTA',
                           2:'ADELANTANDO POR LA DERECHA',3:'ADELANTANDO POR LA IZQUIERDA',
                           11:'GIRANDO O SALIENDO HACIA OTRA VIA O ACCESO POR LA DERECHA',
                           12:'GIRANDO O SALIENDO HACIA OTRA VIA O ACCESO POR LA IZQUIERDA',
                           13:'GIRANDO EN "U"',21:'INCORPORANDOSE DESDE OTRA VIA O ACCESO',
                           22:'CRUZANDO INTERSECCION',23:'ESTACIONANDO O SALIENDO DEL ESTACIONAMIENTO',
                           31:'CIRCULANDO HACIA ATRAS',41:'MANIOBRA SUBITA PARA SALVAR OBSTACULO O VEHICULO',
                           42:'MANIOBRA SUBITA PARA SALVAR PEATON AISLADO O EN GRUPO',
                           43:'BRUSCA REDUCCION DE VELOCIDAD',
                           51:'RETENCION POR IMPERATIVO DE LA CIRCULACION',52:'PARADO O ESTACIONADO',
                           61:'FUGADO',71:'OTRA',72:'SE IGNORA'},

    "INFRACC_VELOCIDAD": {0:'SIN DATO',
             1:'VELOCIDAD INADECUADA PARA LAS CONDICIONES EXISTENTES',
             2:'SOBREPASAR LA VELOCIDAD ESTABLECIDA',
             3:'MARCHA LENTA ENTORPECIENDO LA CIRCULACION',
             4:'NINGUNA',5:'SE IGNORA'},


    "INFRACC_PEATON": {0:'SIN DATO',1:'NO RESPETAR SEÑAL DE SEMAFORO',
                      2:'NO UTILIZAR PASO PARA PEATONES',3:'NO RESPETAR SEÑAL DE AGENTE',
                      4:'IRRUMPIR O CRUZAR LA VIA ANTIRREGLAMENTARIAMENTE',
                      5:'ESTAR O MARCHAR POR LA CALZADA EN FORMA ANTIRREGLAMENTARIA',
                      6:'ESTAR O MARCHAR POR EL ARCEN EN FORMA ANTIRREGLAMENTARIA',
                      7:'SUBIR O BAJAR DEL VEHICULO ANTIRREGLAMENTARIAMENTE',8:'OTRAS INFRACCIONES',
                      9:'NINGUNA INFRACCION'}
         }

df_all_years.replace(Decoder, inplace=True)
df_all_years.head()
df_all_years.tail()
            
print("CREADA VISTA CON DATOS DE PERSONAS DE 2008 A 2015")
df_all_years.to_csv("./cleaned/view_pers_08_15.csv",index=False,sep=";",encoding="ISO-8859-1")
df_all_years=pd.read_csv("./cleaned/view_pers_08_15.csv",delimiter=";",encoding="ISO-8859-1",dtype={'ID_ACCIDENTE': object})
#comprobacion
df_all_years.info()
df_all_years.head()
df_all_years.tail()

{'Pers': ['ID_ACCIDENTE',
  'ID_VEHICULO',
  'ID_PERSONA',
  'ID_CONDUCTOR',
  'ID_PASAJERO',
  'ID_PEATON',
  'EDAD',
  'COD_SEXO',
  'ANIO_PERMISO',
  'MANIOBRAS',
  'INFRACC_VELOCIDAD',
  'INFRACC_PEATON',
  'ANIO']}

1     1042224
2      573607
3       78297
4       19402
5        5733
6        2203
7        1023
8         503
9         269
10        188
11        149
13        102
12         99
14         67
16         60
15         54
19         50
20         48
21         46
43         44
22         44
18         42
27         38
23         37
17         37
24         29
25         29
26         26
38         23
31         22
       ...   
91          1
82          1
75          1
76          1
81          1
83          1
84          1
85          1
73          1
87          1
88          1
89          1
92          1
69          1
71          1
70          1
68          1
67          1
66          1
65          1
64          1
63          1
62          1
61          1
60          1
59          1
58          1
57          1
56          1
93          1
Name: ID_VEHICULO, dtype: int64

1    1214662
2     412117
3      97957
Name: ID_PERSONA, dtype: int64

1     1355332
2      312839
3       40573
4       10100
5        2939
6        1145
7         538
8         287
9         168
10        109
11         81
12         63
13         47
14         41
16         33
15         32
18         26
19         25
20         25
21         23
17         22
27         22
22         21
23         19
24         15
25         13
26         13
28         12
29         11
30         10
       ...   
79          1
80          1
81          1
82          1
83          1
84          1
85          1
86          1
87          1
88          1
89          1
74          1
73          1
72          1
63          1
57          1
58          1
59          1
60          1
61          1
62          1
64          1
71          1
65          1
66          1
67          1
68          1
69          1
70          1
0           1
Name: ID_CONDUCTOR, dtype: int64

1      1554820
2        89740
3        41701
4        18324
5         7140
6         3692
7         2043
8         1211
9          753
10         544
11         427
12         357
13         310
14         268
15         243
16         220
17         203
18         189
19         172
20         157
21         149
22         136
23         125
24         119
25         111
26         104
27          94
28          88
29          85
30          82
        ...   
91           1
92           1
93           1
94           1
95           1
103          1
97           1
98           1
99           1
100          1
101          1
102          1
88           1
89           1
78           1
87           1
86           1
85           1
84           1
83           1
82           1
81           1
80           1
79           1
77           1
76           1
75           1
74           1
73           1
104          1
Name: ID_PASAJERO, dtype: int64

...

1     468748
52    276890
21    220281
22    169912
72    135879
12     94962
15     77230
16     74998
71     27395
51     25886
3      24421
11     18451
2      12843
77     11172
0       8077
41      7491
43      7381
19      7090
23      7016
20      6090
24      5915
17      5396
31      4375
18      4215
13      3546
5       3312
61      2137
25      1794
6       1792
14      1765
29      1723
7       1588
8       1269
42       967
30       716
4        609
9        593
26       509
10       179
27       123
Name: MANIOBRAS, dtype: int64

3    857098
4    644167
1     85123
5     82347
0     50616
2      5385
Name: INFRACC_VELOCIDAD, dtype: int64

2    1063576
9     628042
1      15548
4       6812
0       5197
3       2493
8       2267
5        652
6         92
7         57
Name: INFRACC_PEATON, dtype: int64

2015    238476
2008    223461
2014    222722
2013    219061
2009    212510
2010    206498
2012    202804
2011    199204
Name: ANIO, dtype: int64

CREADA VISTA CON DATOS DE PERSONAS DE 2008 A 2015


#                           FINAL DEL PROCESO 

,ID_ACCIDENTE,ID_VEHICULO,ID_PERSONA,ID_CONDUCTOR,ID_PASAJERO,ID_PEATON,EDAD,COD_SEXO,ANIO_PERMISO,MANIOBRAS,INFRACC_VELOCIDAD,INFRACC_PEATON,ANIO
0,2008010000001,1,2,1,1,1,32,M,9999,21,4,9,2008
1,2008010000001,2,2,1,1,1,60,M,9999,21,4,9,2008
2,2008010000001,2,2,1,2,1,23,V,9999,21,4,9,2008
3,2008010000001,1,2,1,2,1,28,M,9999,21,4,9,2008
4,2008010000001,2,1,2,1,1,61,V,1999,1,5,9,2008


### -------------------------------------------------------------------------------------------------------
### PROCEDO A CARGAR EN DATAFRAMES DE CADA UNO DE LOS FICHEROS DEL DICCIONARIO
### -------------------------------------------------------------------------------------------------------

In [ ]:
import sqlite3 
import pandas as pd
with sqlite3.connect("./cleaned/jj.db") as con:
    df = pd.read_csv('./cleaned/view_vehic_08_15.csv',delimiter=";",encoding="ISO-8859-1")
    c = con.cursor()
    c.executescript('''
    CREATE TABLE f (
    ID_ACCIDENTE INTEGER,
    ID_VEHICULO INTEGER,
    ANIO_MATRICULA_VEHICULO INTEGER,
    MES_MATRICULA_VEHICULO INTEGER,
    TIPO_VEHICULO INTEGER,
    NUMERO_OCUPANTES_VEH INTEGER,
    ANIO INTEGER)''')
  
    df.to_sql('f', con, index=False,schema=False)
    con.commit

In [ ]:
import sqlite3
with sqlite3.connect("./cleaned/datos_dgt.db") as con:
    c = con.cursor()
    c.executescript(
    "CREATE TABLE Direcciones (calle TEXT, cp NUMERIC);"
    "INSERT INTO Direcciones VALUES ('Calle 1', 1644);"
    "INSERT INTO Direcciones VALUES ('Calle 2', 7594);"
    )
    c.executescript(
    "CREATE TABLE vehic;"
    )
    df = pandas.read_csv('./cleaned/view_vehic_08_15.csv',delimiter=";",encoding="ISO-8859-1")
    df.to_sql(vehic, con, index=False)
    con.commit
c.close

In [ ]:
#guardo las claves de cada tipo de fichero
list_keys_dic=dic_all_files.keys()
list_keys_dic
#Actualizo el diccionario con la rutas a los ficheros de trabajo
dic_all_files['Accvict'].append('./cleaned/view_accvict_08_15.csv')
dic_all_files['Pers'].append('./cleaned/view_pers_08_15.csv')
dic_all_files['Vehic'].append('./cleaned/view_vehic_08_15.csv')
dic_all_files

In [ ]:
dic_all_files['Accvict'][8]


In [ ]:
#Creo un dataframe vacio para posteriormente rellenarlo con los años 2001 al 2015
#Recorriendo por cada key los n values del diccionario

dic_total={}
df_total=pd.DataFrame()

for key in list_keys_dic:
    print ("#### "+key+" ####")
    for file in dic_all_files[key]:
        #meto en df cada file quedandome solo con las columnas seleccionadas anteriormente
        df_year=pd.read_csv(file,delimiter=";",encoding="ISO-8859-1",usecols=dic_all_headers[key])
        df_year=df_year.fillna(0)
        print("Cargando fichero-->"+file+"\n\t"+"Registros= "+str(df_year.shape[0])+"\n\t"+"Columnas= "+str(df_year.shape[1]))
        #df_total=df_total.append(df_year)
        df_total=pd.concat([df_total,df_year],ignore_index=True)
    dic_total[key] = df_total
print ("\n\t\t"+"##### CARGADOS EN DF_TOTAL TODOS LOS PERIODOS ########"+"\t\t")
df_total.shape

#### AHORA TENGO DE DATA FRAMES CON TODOS LOS AÑOS DESDE 2008 HASTA 2015

In [ ]:
##para categorizar
#dic_total['Accvict']['HORA']=dic_total['Accvict']['HORA'].astype('category')
#dic_total['Accvict']['HORA'].describe()
#este funciona
dic_total['Accvict']['HORA']= pd.Series(pd.Categorical(dic_total['Accvict']['HORA'], categories=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]))
dic_total['Accvict']['HORA'].cat.rename_categories(['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23'])
dic_total['Accvict']['HORA'].cat.codes

In [ ]:
#Extraigo los datos cuantitativos de victimas por
# 1. Año-mes-dia de la semana
# 2. Localizacion geografica

accvict_geo=pd.get_dummies(dic_total['Accvict'], prefix=['CA','PROV'], columns=['COMUNIDAD_AUTONOMA','PROVINCIA'])


In [ ]:
dic_total['Accvict']

In [104]:
df = pd.DataFrame({'A': ['x', 'y', 'x'], 'B': ['z', 'u', 'z'],
                  'C': [1,2,3],
                  'D':['j', 'l', 'j']})

In [105]:
df

,A,B,C,D
0,x,z,1,j
1,y,u,2,l
2,x,z,3,j


In [106]:
df_1= pd.get_dummies(df)
df_1

,C,A_x,A_y,B_u,B_z,D_j,D_l
0,1,1,0,0,1,1,0
1,2,0,1,1,0,0,1
2,3,1,0,0,1,1,0


In [107]:
df_2=pd.get_dummies(df, prefix=['DDDD','DDD','D'], columns=['A','B','C'])
df_2

,D,DDDD_x,DDDD_y,DDD_u,DDD_z,D_1,D_2,D_3
0,j,1,0,0,1,1,0,0
1,l,0,1,1,0,0,1,0
2,j,1,0,0,1,0,0,1


In [ ]:
  df_year=pd.read_csv(files_list[file],delimiter=";",encoding="ISO-8859-1",converters={'COD_MUNICIPIO': str, 'MUNICIPIO': str})


### LECTURA DE LOS DATASET DE VEHICULOS PARA LOS AÑOS 2011-2015

In [ ]:
df_year.shape[0]

In [ ]:
#Creo un dataframe vacio para posteriormente rellenarlo con los años 2001 al 2015
df_accvict_08_15 = pd.DataFrame()
dict={}
for key in list_keys_dic:
    for file in range(0,len(files_list)):
        df_year=pd.read_csv(files_list[file],delimiter=";",encoding="ISO-8859-1")
        df_year.fillna(0)
        dict[tipology]=files_list[file]
        print("Cargando fichero-->"+files_list[file]+"\n\t"+"Registros= "+str(df_year.shape[0])+"\n\t"+"Columnas= "+str(df_year.shape[1]))
        if tipology=='list_accvict'
        %(tipology)+_08_15=df_accvict_08_15.append(df_year)


In [ ]:
j=df_accvict_08_15.groupby(['ANIO']).size()


In [ ]:
type(j)

In [ ]:
df_accvict_08_15.groupby(['ANIO']).agg(['mean', 'count'])

In [ ]:
print(df_accvict_08_15.head())
#df1=df[['ANIO','TOT_VICTIMAS']]


In [ ]:
dict

In [ ]:
threedee.scatter(df.ANIO, df["TOT_MUERTOS"], df['Volume'])